### Hypothesis Testing

### ABTest Guide
https://abtestguide.com/abtestsize/

In [2]:
%load_ext rpy2.ipython

R[write to console]: During startup - 
R[write to console]: Warning messages:

R[write to console]: 1: Setting LC_COLLATE failed, using "C" 

R[write to console]: 2: Setting LC_TIME failed, using "C" 

R[write to console]: 3: Setting LC_MESSAGES failed, using "C" 

R[write to console]: 4: Setting LC_MONETARY failed, using "C" 



In [81]:
%%R

calculate_power <- function(p1, p2, power, sig.level) {
    a = power.prop.test(p1 = p1, 
                        p2 = p2, 
                        power, 
                        n = NULL, 
                        sig.level = sig.level,
                        alternative = "two.sided")
    print(a)
    a['n']
}

calculate_power(0.01, 0.012, 0.80, 0.05)


     Two-sample comparison of proportions power calculation 

              n = 42692.8
             p1 = 0.01
             p2 = 0.012
      sig.level = 0.05
          power = 0.8
    alternative = two.sided

NOTE: n is number in *each* group

$n
[1] 42692.8



In [ ]:
### EVAN MILLER / STANFORD
# Evan Miller Source Code: 
# https://gist.github.com/mottalrd/7ddfd45d14bc7433dec2#file-gistfile1-txt

In [12]:
from scipy.stats import norm
import math

In [75]:
## Blog post using engineering statistics link
## http://www.alfredo.motta.name/ab-testing-from-scratch/
## https://www.itl.nist.gov/div898/handbook/prc/section2/prc242.htm

def calculate_sample_size(p1, p2, sig_level, power):
    
    delta = abs(p1 - p2)
    
    z_stat_alpha = norm.ppf(1-((1-sig_level)/2))
    z_stat_beta = norm.ppf(power)
    
    sd1 = math.sqrt(p1*(1-p1))
    sd2 = math.sqrt(p2*(1-p2))
    
    n = ((z_stat_alpha * sd1 + z_stat_beta * sd2)/delta) ** 2
    
    return n

print(calculate_sample_size(0.01, 0.012, 0.95, 0.80))

20542.643493459902


In [76]:
## On Evan Miller Website
## and gist: https://gist.github.com/mottalrd/7ddfd45d14bc7433dec2
def calculate_sample_size(p1, p2, sig_level, power):
    
    delta = abs(p1 - p2)
    
    z_stat_alpha = norm.ppf(1-((1-sig_level)/2))
    z_stat_beta = norm.ppf(power)
    
    sd1 = math.sqrt(2 * p1 * (1-p1))
    sd2 = math.sqrt(p1 * (1-p1) + (p1 + delta) * (1 - p1 - delta))
    
    n = ((z_stat_alpha * sd1 + z_stat_beta * sd2)/delta) ** 2
    
    return n

print(calculate_sample_size(0.01, 0.012, 0.95, 0.80))

39985.96074505791


In [98]:
### Stanford Lecture

def calculate_sample_size(p1, p2, sig_level=0.95, 
                          power=0.80, r=1, tails='one'):
    
    delta = abs(p1 - p2)
    
    if tails == 'one':
        z_stat_alpha = norm.ppf(sig_level)
    elif tails == 'two':
        z_stat_alpha = norm.ppf(sig_level + ((1 - sig_level) / 2))
    
    z_stat_beta = norm.ppf(power)
    
    n = ((r+1)/r) * p1 * (1 - p1) * ((z_stat_beta + z_stat_alpha) ** 2) / (delta**2)
    
    return n

print(calculate_sample_size(0.01, 0.012, r=1))

30603.658298497845


In [118]:
calculate_sample_size(0.04, 0.05, sig_level = 0.90, power = 0.80)

3462.0385804128

In [73]:
## https://towardsdatascience.com/the-math-behind-a-b-testing-with-example-code-part-1-of-2-7be752e1d06f
## claims to take from stanford, but looks like pooled variance is wrong. You have to assume the null hypothesis
## crf KA video: https://www.khanacademy.org/math/ap-statistics/two-sample-inference/two-sample-z-test-proportions/v/hypothesis-test-for-difference-in-proportions?modal=1

def min_sample_size(bcr, mde, power=0.8, sig_level=0.05):
    """
    Returns the minimum sample size to set up a split test
    
    Arguments:
        bcr (float): probability of success for control, sometimes
        referred to as baseline conversion rate
        mde (float): minimum change in measurement between control
        group and test group if alternative hypothesis is true, sometimes
        referred to as minimum detectable effect
        power (float): probability of rejecting the null hypothesis when the
        null hypothesis is false, typically 0.8
        sig_level (float): significance level often denoted as alpha,
        typically 0.05
        
    Returns:
        min_N: minimum sample size (float)
        
    References:
        Stanford lecture on sample sizes
        http://statweb.stanford.edu/~susan/courses/s141/hopower.pdf
    """

    # find Z_beta from desired power
    Z_beta = norm.ppf(power)

    # find Z_alpha
    Z_alpha = norm.ppf(1-sig_level/2)
    print(Z_alpha)

    # average of probabilities from both groups
    pooled_prob = (bcr + bcr+mde) / 2
    print(pooled_prob)

    min_N = (2 * pooled_prob * (1 - pooled_prob) * (Z_beta + Z_alpha)**2
             / mde**2)

    return min_N

In [63]:
min_sample_size(0.01, 0.002)

1.959963984540054
0.011


42693.98131499187

In [ ]:
## http://powerandsamplesize.com/Calculators/Compare-2-Proportions/2-Sample-Equality
## why is this method using the variance from p(B) if we're supposed to assume null hypothesis is true

In [78]:
# https://select-statistics.co.uk/calculators/sample-size-calculator-two-proportions/

In [79]:
# http://www2.ccrb.cuhk.edu.hk/stat/proportion/Casagrande.htm

In [119]:
# from work notebook

In [121]:
import numpy as np
import pandas as pd

from scipy import stats
from scipy.stats import chi2_contingency
from scipy.stats import ttest_ind, ttest_rel, f_oneway, mannwhitneyu, wilcoxon

import statsmodels.stats.api as sm
from statsmodels.stats.power import TTestIndPower
from statsmodels.stats.proportion import proportions_ztest
import numpy as np
import pandas as pd

In [122]:
class TwoPropZTest():
    
    def __init__(self, control_n, control_conversions, variant_n, variant_conversions):
        self.control_n = control_n
        self.control_conversions = control_conversions
        self.variant_n = variant_n
        self.variant_conversions = variant_conversions
        
        self.control_cr = self.control_conversions/self.control_n
        self.variant_cr = self.variant_conversions/self.variant_n
        
        self.results = {}
        
        
    def z_score(self):
        
        control_se = np.sqrt((self.control_cr*(1-self.control_cr)/self.control_n))
        variant_se = np.sqrt((self.variant_cr*(1-self.variant_cr)/self.variant_n))
        se_diff = np.sqrt(np.power(control_se, 2) + np.power(variant_se, 2))

        z_score = (self.variant_cr-self.control_cr)/se_diff
        
        self.results['z_score'] = z_score
        
        # alternatively
        # z_statistic = stats.zscore(array1-array2) ?
        
        return z_score
    
    def p_value(self):
        
        z_score = self.z_score()
        p_value = 1-stats.norm.cdf(z_score)
        
        self.results['p_value'] = p_value
        
        return p_value
    
    def power(self, alpha, tail='larger'):
        
        # check: http://powerandsamplesize.com/Calculators/Compare-2-Proportions/2-Sample-1-Sided
        # matches R's pwr library
        
        nobs2 = self.variant_n
        ratio = self.variant_n/self.control_n
        
        es = sm.proportion_effectsize(self.variant_cr, self.control_cr) #swapped due to weird sm implementation
        
        power_analysis = sm.NormalIndPower(0)
        power = power_analysis.solve_power(power=None, effect_size=es, 
                                           nobs1=self.control_n, ratio=ratio,
                                           alpha=alpha, alternative=tail)
        
        self.results['power'] = power
        
        return power

In [128]:
def TwoPropZTest_sample_size(ex_control_cr, ex_variant_cr, alpha, power, tail='larger'):
    
    """
    Returns PER GROUP numbers. To get total sample size, multiply by 2
    """
    
    es = sm.proportion_effectsize(ex_variant_cr, ex_control_cr)
    
    test = sm.NormalIndPower(0)
    sample_size = test.solve_power(nobs1=None, ratio=1, effect_size=es, alpha=alpha, power=power, alternative=tail)
    
    return sample_size

In [123]:
class FisherExact():
    
    def __init__(self, control_n, control_conversions, variant_n, variant_conversions):
        
        self.control_n = control_n
        self.control_conversions = control_conversions
        self.variant_n = variant_n
        self.variant_conversions = variant_conversions
        self.results = {}

    def generate_contingency_table(self, control_n, control_conversions, variant_n, variant_conversions):

        control_non = control_n - control_conversions
        variant_non = variant_n - variant_conversions

        cont_array = np.array([[control_conversions, control_non],
                               [variant_conversions, variant_non]])
        
        self.contingency_table = pd.DataFrame(cont_array, 
                                              index=['control', 'variant'],
                                              columns=['convert', 'did not convert'])

        return self.contingency_table
    
    def p_value(self, tail='two-sided'):
        
        cont_table = self.generate_contingency_table(self.control_n, self.control_conversions, 
                                                     self.variant_n, self.variant_conversions)
        
        self.odds_ratio, self.p_value = stats.fisher_exact(cont_table, alternative=tail)
        
        self.results['p value'] = self.p_value
        self.results['odds ratio'] = self.odds_ratio
        
        return self.p_value

In [124]:
ab_test = TwoPropZTest(182, 35, 188, 61)
ab_test.z_score()
ab_test.p_value()
ab_test.power(0.05, 'larger')
ab_test.results

{'z_score': 2.9410157224928595,
 'p_value': 0.0016356896438655122,
 'power': 0.8997528382700732}

In [ ]:
TwoPropZTest_sample_size(0.02, 0.023, 0.05, 0.8, 'larger')

In [125]:
ab_test = TwoPropZTest(182, 35, 188, 61)
ab_test.z_score()
ab_test.p_value()
ab_test.power(0.05, 'two-sided')
ab_test.results

{'z_score': 2.9410157224928595,
 'p_value': 0.0016356896438655122,
 'power': 0.8327366597430156}

In [126]:
ab_test = TwoPropZTest(324, 35, 323, 48)
ab_test.z_score()
ab_test.p_value()
ab_test.power(0.05, 'larger')
ab_test.results

{'z_score': 1.5459382713230716,
 'p_value': 0.06105973827573996,
 'power': 0.46132455828958546}

In [129]:
TwoPropZTest_sample_size(0.1, 0.15, 0.05, 0.8, 'larger')

535.9146420344101

In [130]:
test = FisherExact(324, 35, 323, 48)
test.p_value()
print(test.results)
display(test.contingency_table)

{'p value': 0.1280406654595789, 'odds ratio': 0.6938437139561707}


,convert,did not convert
control,35,289
variant,48,275


In [ ]:
# how to find power for fisher test on python?

In [131]:
stat, p, dof, expected = chi2_contingency(table)

stat, p = ttest_ind(data1, data2)
stat, p = ttest_rel(data1, data2)
stat, p = f_oneway(data1, data2)
stat, p = mannwhitneyu(data1, data2)
stat, p = willcoxon(data1, data2)

NameError: name 'table' is not defined

In [132]:
count = np.array([35, 61])
nobs = np.array([182, 186])
stat, pval = proportions_ztest(count, nobs, 0, alternative='larger')
print(stat)
print(1-pval)

-2.96287653203812
0.001523893967572243


In [133]:
effect_size = 0.3757
ratio = 323/324
power_analysis = zt_ind_solve_power(effect_size = effect_size, nobs1=324, alpha=0.05, 
                                    ratio=ratio, alternative='larger')
power_analysis


NameError: name 'zt_ind_solve_power' is not defined

In [134]:
effect_size = 0.8
alpha = 0.05 # significance level
power = 0.8

power_analysis = TTestIndPower()
sample_size = power_analysis.solve_power(effect_size = effect_size, 
                                         power = power, 
                                         alpha = alpha)

print('Required sample size: {0:.2f}'.format(sample_size))

Required sample size: 25.52
